# Finding pattern in text

In [7]:
%load_ext autoreload
%autoreload 2
import time_comparison

### Naive algorithm

In [3]:
def naive_compare(text, pattern):
    res = []
    for s in range(0, len(text) - len(pattern) + 1):
        if (pattern == text[s:(s + len(pattern))]):
            res.append(s)
    return res

### Finite automaton

In [5]:
def get_matching_positions(text, delta):
    res = []
    q = 0
    for s in range(0, len(text)):
        if (text[s] not in delta[q]):
            q = 0
            continue

        q = delta[q][text[s]]
        if (q == len(delta) - 1):
            res.append(s + 1 - q)
    return res

def get_delta(pattern):
    res = []
    letters = set()
    for c in pattern:
        letters.add(c)

    for q in range(0, len(pattern) + 1):
        res.append({})
        for c in letters:
            k = min(len(pattern) + 1, q + 2)
            while k > 0:
                k = k - 1
                if (pattern[0:k] == pattern[(q - k + 1):q] + c):
                    break
            res[q][c] = k

    return res

def finite_automaton_compare(text, pattern):
    return get_matching_positions(text, get_delta(pattern))

### Knuth-Morris-Pratt algorithm

In [10]:
def kmp_compare(text, pattern):
    pi = build_prefix_array(pattern)
    q = 0
    res = []
    for i in range(0, len(text)):
        while (q > 0 and pattern[q] != text[i]):
            q = pi[q - 1]
        if (text[i] == pattern[q]):
            q = q + 1
        if (q == len(pattern)):
            res.append(i - q + 1)
            q = pi[q - 1]
    return res

def build_prefix_array(pattern):
    pi = [0]
    k = 0
    for q in range(1, len(pattern)):
        while (k > 0 and pattern[q] != pattern[k]):
            k = pi[k - 1]
        if (pattern[q] == pattern[k]):
            k = k + 1
        pi.append(k)
    return pi


### Time comparison for long pattern

In [11]:
text, pattern = time_comparison.generate_long_text(5000000)
print("Naive: {}s".format(time_comparison.measure_time(naive_compare, text, pattern)))
print("Finite automaton: {}s".format(time_comparison.measure_time(finite_automaton_compare, text, pattern)))
print("KMP: {}s".format(time_comparison.measure_time(kmp_compare, text, pattern)))

Naive: 0.869530439376831s
Finite automaton: 0.34185051918029785s
KMP: 0.7687621116638184s


### Time comparison for text file

In [59]:
def compare_times(text, pattern):
    naiveTime, naiveRes = time_comparison.measure_time_return(naive_compare, text, pattern)
    print("Naive (time: {}s): {} matches".format(naiveTime, len(naiveRes)))
    
    automatonTime, automatonRes = time_comparison.measure_time_return(finite_automaton_compare, text, pattern)
    print("Finite automaton: (time: {}s): {} matches".format(automatonTime, len(automatonRes)))
    
    kmpTime, kmpRes = time_comparison.measure_time_return(kmp_compare, text, pattern)
    print("KMP: (time: {}s): {} matches".format(kmpTime, len(kmpRes)))

In [60]:
pattern = "art"

with open("data/text1.txt", "r") as file:
    text = file.read()
    compare_times(text, pattern)

Naive (time: 0.09780764579772949s): 273 matches
Finite automaton: (time: 0.02242279052734375s): 273 matches
KMP: (time: 0.039588212966918945s): 273 matches


### Time comparison for Wikipedia dump

In [61]:
pattern = "kruszwil"

with open("data/wikipedia-tail-kruszwil.txt") as file:
    text = file.read()
    compare_times(text, pattern)

Naive (time: 60.14912438392639s): 13 matches
Finite automaton: (time: 51.40565085411072s): 13 matches
KMP: (time: 61.85614228248596s): 13 matches


## Special Patterns

Text and pattern for which naive algorithm's running time will be at least two times longer than finite automaton and KMP running time. 

The idea here is to construct a text which contains the pattern multiple times, shifted by one character (because both KMP and finite automaton algorithms look at the longest prefix which is also the suffix of the pattern, they do not compare all of the characters with the pattern in the current window).

In [57]:
def multiply_s(n, pat):
    s = ""
    for i in range(0, n):
        s = s + pat
    return s

In [58]:
ab_text = multiply_s(10000000, "a")
compare_times(ab_text, multiply_s(10000, "a"))

Naive (time: 6.526066780090332s): 9990001
Finite automaton: (time: 3.5208828449249268s): 9990001
KMP: (time: 4.21057391166687s): 9990001


Pattern for which preprocessing time of finite automaton is twice as long as KMP preprocessing time.

The idea is to construct a pattern does not have a matching prefix and suffix. During construction of finite automaton we naively check for longest prefx-suffix, while during construction of pi array in KMP we use previously computed values.

Example pattern: "abcdefghijkl"